# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> APCOMP 295 Advanced Practical Data Science
## Exercise 5: Semantic Segmentation with Transfer Learning



**Harvard University**<br/>
**Fall 2020**<br/>
**Instructors**: Pavlos Protopapas


<hr style="height:2pt">

**Each assignment is graded out of 5 points.  The topic for this assignment is Semantic Segmentation with Transfer Learning.**

**Due:** 10/20/2020 10:15 AM EDT

**Submit:** We won't be re running your notebooks, please ensure output is visible in the notebook.

#### Learning Objectives

In this exercise you will cover the following topics:  
- Building data input pipelines using `tf.data`
- Build Semantic Segmentation model with a pretrained image classifier as base
- Using callbacks to manage model training paramaters

---

#### Imports

In [ ]:
import os
import requests
import zipfile
import shutil
import json
import time
import sys
import cv2
import numpy as np
import pandas as pd
from glob import glob
from string import Template
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from keras.utils.layer_utils import count_params

from google.colab import auth
from google.cloud import storage

from sklearn.model_selection import train_test_split

#### Verify Setup

It is a good practice to verify what version of TensorFlow & Keras you are using. Also verify if GPU is enabled and what GPU you have. Run the following cells and record the version of TensorFlow

References:
- [Eager Execution](https://www.tensorflow.org/guide/eager)
- [Data Performance](https://www.tensorflow.org/guide/data_performance)

In [ ]:
# Enable/Disable Eager Execution
# Reference: https://www.tensorflow.org/guide/eager
# TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, 
# without building graphs

#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

print("tensorflow version", tf.__version__)
print("keras version", tf.keras.__version__)
print("Eager Execution Enabled:", tf.executing_eagerly())

# Get the number of replicas 
strategy = tf.distribute.MirroredStrategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

devices = tf.config.experimental.get_visible_devices()
print("Devices:", devices)
print(tf.config.experimental.list_logical_devices('GPU'))

print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("All Physical Devices", tf.config.list_physical_devices())

# Better performance with the tf.data API
# Reference: https://www.tensorflow.org/guide/data_performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

Run this cell to see what GPU you have. If you get a P100 or T4 GPU that's great. If it's K80, it will still work but it will be slow.

In [ ]:
!nvidia-smi

#### Utils

Here are some util functions that will be helpful

In [ ]:
def download_file(packet_url, base_path="", extract=False):
  if base_path != "":
    if not os.path.exists(base_path):
      os.mkdir(base_path)
  packet_file = os.path.basename(packet_url)
  with requests.get(packet_url, stream=True) as r:
      r.raise_for_status()
      with open(os.path.join(base_path,packet_file), 'wb') as f:
          for chunk in r.iter_content(chunk_size=8192):
              f.write(chunk)
  
  if extract:
    with zipfile.ZipFile(os.path.join(base_path,packet_file)) as zfile:
        zfile.extractall(base_path)

class JsonEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, decimal.Decimal):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return super(JsonEncoder, self).default(obj)

def save_model(path="models",model_name="model01"):

  # Ensure path exists
  if not os.path.exists(path):
      os.mkdir(path)

  # Save the enitire model (structure + weights)
  model.save(os.path.join(path,model_name+".hdf5"))

  # Save only the weights
  model.save_weights(os.path.join(path,model_name+".h5"))

  # Save the structure only
  model_json = model.to_json()
  with open(os.path.join(path,model_name+".json"), "w") as json_file:
      json_file.write(model_json)

def get_model_size(path="models",model_name="model01"):
  model_size = os.stat(os.path.join(path,model_name+".hdf5")).st_size
  return model_size

def evaluate_save_model(model,test_data, training_results,execution_time, learning_rate, batch_size, epochs, optimizer,save=True):
    
  # Get the model train history
  model_train_history = training_results.history
  # Get the number of epochs the training was run for
  num_epochs = len(model_train_history["loss"])

  # Plot training results
  fig = plt.figure(figsize=(15,5))
  axs = fig.add_subplot(1,2,1)
  axs.set_title('Loss')
  # Plot all metrics
  for metric in ["loss","val_loss"]:
      axs.plot(np.arange(0, num_epochs), model_train_history[metric], label=metric)
  axs.legend()
  
  axs = fig.add_subplot(1,2,2)
  axs.set_title('Accuracy')
  # Plot all metrics
  for metric in ["accuracy","val_accuracy"]:
      axs.plot(np.arange(0, num_epochs), model_train_history[metric], label=metric)
  axs.legend()

  plt.show()
  
  # Evaluate on test data
  evaluation_results = model.evaluate(test_data)
  print(evaluation_results)
  
  if save:
      # Save model
      save_model(model_name=model.name)
      model_size = get_model_size(model_name=model.name)

      # Save model history
      with open(os.path.join("models",model.name+"_train_history.json"), "w") as json_file:
          json_file.write(json.dumps(model_train_history,cls=JsonEncoder))

      trainable_parameters = count_params(model.trainable_weights)
      non_trainable_parameters = count_params(model.non_trainable_weights)

      # Save model metrics
      metrics ={
          "trainable_parameters":trainable_parameters,
          "execution_time":execution_time,
          "loss":evaluation_results[0],
          "accuracy":evaluation_results[1],
          "model_size":model_size,
          "learning_rate":learning_rate,
          "batch_size":batch_size,
          "epochs":epochs,
          "optimizer":type(optimizer).__name__,
          "label_names": label_names,
          "name": model.name,
          "id": int(time.time())
      }
      with open(os.path.join("models",model.name+"_metrics.json"), "w") as json_file:
          json_file.write(json.dumps(metrics,cls=JsonEncoder))

## Question 1 : Setup & Concepts (1.0 Point)

#### a) Explain in your own words the difference between image classification and semantic segmentation

Needs to be 2-3 sentences at most.
 

##### **Submission:** 

*your answer here*

#### b) How is this difference reflected in the architecture of the neural network used for either classification and segmentation ?

what is the shape of the last layer for each?  


##### **Submission:** 

*your answer here*

## Download & Prepare Data

#### Download

In [ ]:
start_time = time.time()
download_file("https://github.com/shivasj/dataset-store/releases/download/v2.0/pascal_voc2012_subset.zip", base_path="datasets", extract=True)
execution_time = (time.time() - start_time)/60.0
print("Download execution time (mins)",execution_time)

#### Explore

In [ ]:
label_names = ['background', 'aeroplane', 'car', 'cat', 'dog', 'person']
# Number of unique labels
num_classes = len(label_names) 
# Create label index for easy lookup
label2index = dict((name, index) for index, name in enumerate(label_names))
index2label = dict((index, name) for index, name in enumerate(label_names))

# Input Images
data_x = glob('datasets/inputs/*.jpg')
print("Dataset count:",len(data_x))
print(data_x[:5])

# Output masks
data_y = [x.replace("inputs","outputs").replace("jpg","png") for x in data_x]
print("Mask files count:",len(data_y))
print(data_y[:5])

#### View Images

In [ ]:
# Generate a random sample of index
image_samples = np.random.randint(0,high=len(data_x)-1, size=4)

fig = plt.figure(figsize=(20,8))
for i,img_idx in enumerate(image_samples):
  axs = fig.add_subplot(2,4,i+1)
  # Read image
  sample_input = cv2.imread(data_x[img_idx])
  plt.imshow(cv2.cvtColor(sample_input, cv2.COLOR_BGR2RGB))
  plt.axis('off')

  sample_mask = cv2.imread(data_y[img_idx])
  sample_mask = sample_mask[:,:,0]
  print("Mask Shape:",sample_mask.shape,"Classes:",np.unique(sample_mask))
  axs = fig.add_subplot(2,4,i+1+4)
  plt.imshow(sample_mask)
  plt.axis('off')

  title = ""
  for cls in np.unique(sample_mask):
    if cls > 0:
      title = title + index2label[cls] +", "
  axs.set_title(title)

plt.show()

In [ ]:
# Find class counts
class_counts = dict((index, 0) for index, name in enumerate(label_names))

for y in data_y:
  mask = cv2.imread(y)
  mask = mask[:,:,0]
  for cls in np.unique(mask):
    class_counts[cls] = class_counts[cls] +1

print("Labels:", label2index)
print("Class Counts:",class_counts)

#### c) Explain how the data for the mask files are stored 

Hint: Analyze the content of `outputs` folder in your dataset 

##### **Submission:** 

*your answer here*

#### Build Data Pipelines

In [ ]:
validation_percent = 0.15

# Split data into train / validate
train_x, validate_x, train_y, validate_y = train_test_split(data_x, data_y, test_size=validation_percent)

print("train_x count:",len(train_x))
print("validate_x count:",len(validate_x))

In [ ]:
image_width = 224
image_height = 224
num_channels = 3
num_output_masks = num_classes
batch_size = 4
train_buffer_size=800
validation_buffer_size=200

# Load Image
def load_image(input_path, mask_path):
  # Input image
  input_image = tf.io.read_file(input_path)
  input_image = tf.image.decode_jpeg(input_image, channels=num_channels)
  
  # Mask image
  mask_image = tf.io.read_file(mask_path)
  mask_image = tf.image.decode_png(mask_image, channels=1)

  # Random cropping
  crop_x = np.random.randint(256 - image_width)
  crop_y = np.random.randint(256 - image_height)
  input_image = input_image[crop_y:crop_y+image_height, crop_x:crop_x+image_width]
  mask_image = mask_image[crop_y:crop_y+image_height, crop_x:crop_x+image_width]
  mask_image = tf.cast(mask_image, tf.float32)

  # Random Flipping
  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    mask_image = tf.image.flip_left_right(mask_image)
  
  return input_image, mask_image

# Scale pixels
def scale_image_pixels(input_image, mask_image):
  input_image = tf.cast(input_image, tf.float32)
  input_image = input_image/127.5
  input_image -= 1.
  return input_image, mask_image

# Create TF Dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
validation_data = tf.data.Dataset.from_tensor_slices((validate_x, validate_y))

#############
# Train data
#############
# Apply all data processing logic
train_data = train_data.map(load_image, num_parallel_calls=AUTOTUNE)
train_data = train_data.map(scale_image_pixels, num_parallel_calls=AUTOTUNE)
train_data = train_data.shuffle(buffer_size=train_buffer_size)
train_data = train_data.batch(batch_size)

##################
# Validation data
##################
# Apply all data processing logic
validation_data = validation_data.map(load_image, num_parallel_calls=AUTOTUNE)
validation_data = validation_data.map(scale_image_pixels, num_parallel_calls=AUTOTUNE)
validation_data = validation_data.shuffle(buffer_size=validation_buffer_size)
validation_data = validation_data.batch(batch_size)

print("train_data",train_data)
print("validation_data",validation_data)

---

## Question 2 : Train Semantic Segmentation Model from Scratch (1.5 Point)

#### a) Train a model without transfer learning

- Build a Semantic Segmentation Model. Ensure that the model is built without transfer learning.
- Ensure there is a plot of your training history

#### b) View results from model

## View Results

In [ ]:
cmap_ref = cm.get_cmap('tab20', 12)
cmap_seg = np.zeros((6, 4))
cmap_seg[0] = [0.7, 0.7, 0.7, 0]
for i in range(1, 6):
  cmap_seg[i] = cmap_ref(i)

cmap_seg = ListedColormap(cmap_seg)
print("\nClasses to detect, with corresponding colors:")
plt.imshow([[0, 1, 2, 3, 4, 5]], cmap=cmap_seg)
plt.xticks([0,1 ,2 ,3 ,4 ,5, 6], label_names, rotation=45)
plt.show()

def preprocess_prediction_image(file_path):
  # Read image
  image = cv2.imread(file_path)
  # convert to rgb
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # Reshape
  #image = cv2.resize(image, (image_height, image_width), interpolation=cv2.INTER_AREA)
  crop_x = (image.shape[1] - image_width) // 2
  crop_y = (image.shape[0] - image_height) // 2
  image = image[crop_y:crop_y+image_height, crop_x:crop_x+image_width]
  # Scale pixel
  image = image.astype(np.float32)
  image /= 127.5
  image -= 1.

  return image

In [ ]:
# Generate a random sample of index
image_samples = np.random.randint(0,high=len(validate_x)-1, size=4)

fig = plt.figure(figsize=(20,10))
for i,img_idx in enumerate(image_samples):
  axs = fig.add_subplot(3,4,i+1)
  # Read image
  sample_input = cv2.imread(validate_x[img_idx])
  plt.imshow(cv2.cvtColor(sample_input, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  axs.set_title("Image")

  sample_mask = cv2.imread(validate_y[img_idx])
  sample_mask = sample_mask[:,:,0]
  axs = fig.add_subplot(3,4,i+1+4)
  plt.imshow(sample_mask)
  plt.axis('off')
  axs.set_title("Masks")

  axs = fig.add_subplot(3,4,i+1+8)
  plt.imshow(cv2.cvtColor(sample_input, cv2.COLOR_BGR2RGB))
  plt.axis('off')

  predict_x = [preprocess_prediction_image(validate_x[img_idx])]
  predict_x = np.asarray(predict_x)
  # Make predictions
  predictions = model.predict(predict_x)

  sample_pred = predictions[0]
  # Generate mask outputs
  pred_mask = tf.argmax(sample_pred, axis=-1)
  plt.imshow(pred_mask, cmap=cmap_seg, vmin=0, vmax=num_output_masks, alpha=0.6)

  title = ""
  for cls in np.unique(pred_mask):
    if cls > 0:
      title = title + index2label[cls] +", "
  axs.set_title(title)

plt.show()

#### c) Explain why this model is performing poorly


##### **Submission:** 

*your answer here*

---

## Question 3 : Train Semantic Segmentation Model with Transfer Learning (2.5 Point)

#### a) Train model with transfer learning

- Build a Semantic Segmentation Model but this time with transfer learning.
- Ensure there is a plot of your training history
- Your baseline loss for the models should be a **Validation Loss** less than 0.45

#### b) View results from model

#### c) Explain why this model is doing a "good job" on semantic segmentation

##### **Submission:** 

*your answer here*

#### d) Improve model performance

The dataset is small for this problem and there are a few techniques you could use to further improve model performance. Try a few from this list to see if you can lower your validation loss:
- Change Optimizer, learning rate, loss function etc.
- Using a Learning rate scheduler
- Using Early Stopping to stop training
- Add more data augmentation
- Or any other parameter or metric that works for you

What was the lowest validation loss you were able to achieve? And what techniques did you use and what helped and what did not? (2-3 sentences)

##### **Submission:** 
*your answer here*